# INTRODUCTION TO TENSORBOARD
## The TensorBoard Visualization Framework


<img src='images/tensorboard-the-loop-of-progress.jpg' />


__Change the backend to 'tensorflow' in /home/vagrant/.keras/keras.json__

In [1]:
import keras

Using TensorFlow backend.


In [ ]:
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000                                              
max_len = 500                                                    

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


## TRAINING THE MODEL WITH A TENSORBOARD CLASSBACK

In [ ]:
# Log files will be written at this location.
# Records activation histograms every 1 epoch
# Records embedding data every 1 epoch
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='/home/vagrant/datascience/tf_log',                  
        histogram_freq=1,                      
        embeddings_freq=1,                     
    )
]
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 88s 4ms/step - loss: 0.6367 - acc: 0.6545 - val_loss: 0.4167 - val_acc: 0.8348
Epoch 2/20
20000/20000 [==============================] - 89s 4ms/step - loss: 0.4224 - acc: 0.8267 - val_loss: 0.4336 - val_acc: 0.8270
Epoch 3/20
20000/20000 [==============================] - 88s 4ms/step - loss: 0.3829 - acc: 0.8096 - val_loss: 0.4787 - val_acc: 0.8000
Epoch 4/20
20000/20000 [==============================] - 89s 4ms/step - loss: 0.3743 - acc: 0.7709 - val_loss: 0.5148 - val_acc: 0.7738
Epoch 5/20
20000/20000 [==============================] - 90s 4ms/step - loss: 0.3213 - acc: 0.7449 - val_loss: 0.5959 - val_acc: 0.6718
Epoch 6/20
20000/20000 [==============================] - 95s 5ms/step - loss: 0.2817 - acc: 0.6844 - val_loss: 0.6403 - val_acc: 0.6476
Epoch 7/20
20000/20000 [==============================] - 88s 4ms/step - loss: 0.2536 - acc: 0.6417 - val_loss: 0.7022 - val_acc:

In [ ]:
! tensorboard --logdir=/home/vagrant/datascience/tf_log

You can then browse to http://localhost:6006 and look at your model training. In addition to live graphs of the training and validation metrics, you get access to the Histograms tab, where you can find pretty visualizations of histograms of activation values taken by your layers.

<img src='./images/tf_metrics_monitoring.jpg' />

<img src='./images/tf_activation_histograms.jpg' />

The Embeddings tab gives you a way to inspect the embedding locations and spatial relationships of the 10,000 words in the input vocabulary, as learned by the initial Embedding layer. Because the embedding space is 128-dimensional, TensorBoard automatically reduces it to 2D or 3D using a dimensionality-reduction algorithm of your choice: either principal component analysis (PCA) or t-distributed stochastic neighbor embedding (t-SNE). In the figure below, in the point cloud, you can clearly see two clusters: words with a positive connotation and words with a negative connotation. The visualization makes it immediately obvious that embeddings trained jointly with a specific objective result in models that are completely specific to the underlying task—that’s the reason using pretrained generic word embeddings is rarely a good idea.

<img src='./images/3D_word_embeddings.jpg' />

The Graphs tab shows an interactive visualization of the graph of low-level TensorFlow operations underlying your Keras model. As you can see, there’s a lot more going on than you would expect. The model you just built may look simple when defined in Keras—a small stack of basic layers—but under the hood, you need to construct a fairly complex graph structure to make it work. A lot of it is related to the gradient-descent process. This complexity differential between what you see and what you’re manipulating is the key motivation for using Keras as your way of building models, instead of working with raw TensorFlow to define everything from scratch. Keras makes your workflow dramatically simpler.

<img src='./images/tf_graph_visualization.jpg' />

## SUMMMARY

Keras callbacks provide a simple way to monitor models during training and automatically take action based on the state of the model.

When you’re using TensorFlow, TensorBoard is a great way to visualize model activity in your browser. You can use it in Keras models via the TensorBoard callback.